In [19]:
import numpy as np
import cv2

In [20]:
def merge_regions(image, mask, x, y, width, height):
    # Объединяем регионы путем установки их значений в среднее значение яркости
    region_mean = np.mean(image[y:y+height, x:x+width])
    image[y:y+height, x:x+width] = region_mean
    
    # Отмечаем область региона, как обработанную
    mask[y:y+height, x:x+width] = 255

In [21]:
def should_merge(image, mask, x, y, width, height):
    # Проверяем, должны ли регион быть объединенными
    region = image[y:y+height, x:x+width]
    region_mean = np.mean(region)
    
    # Проверяем, что весь регион имеет близкое к среднему значение яркости
    if np.allclose(region, region_mean):
        # Проверяем, что регион еще не был обработан
        if np.all(mask[y:y+height, x:x+width] == 0):
            return True
    
    return False

In [22]:
def split_and_merge_recursive(image, mask, x, y, width, height, min_region_size):
    # Разделение региона
    if width * height > min_region_size:
        # Разделяем регион пополам по вертикали
        if width > height:
            mid = x + width // 2
            split_and_merge_recursive(image, mask, x, y, mid - x, height, min_region_size)
            split_and_merge_recursive(image, mask, mid, y, width - (mid - x), height, min_region_size)
        # Разделяем регион пополам по горизонтали
        else:
            mid = y + height // 2
            split_and_merge_recursive(image, mask, x, y, width, mid - y, min_region_size)
            split_and_merge_recursive(image, mask, x, mid, width, height - (mid - y), min_region_size)

    # Объединение региона, если требуется
    if should_merge(image, mask, x, y, width, height):
        merge_regions(image, mask, x, y, width, height)

In [23]:
def split_and_merge(image, min_region_size):
    # Создаем маску для отслеживания уже обработанных регионов
   
    
    mask = np.zeros(image.shape[:2], dtype=np.uint8)
    # Вызываем функцию разделения и объединения регионов
    split_and_merge_recursive(image, mask, 0, 0, image.shape[1], image.shape[0], min_region_size)
    return image

In [24]:
image = cv2.imread('circles.jpg', cv2.IMREAD_GRAYSCALE)
image = split_and_merge(image, min_region_size=10)
cv2.imwrite('segmented_image.jpg', image)

True